## Quadratic Programming: Five Problems with Three Starting Points

### Quadratic Programming (QP)

Quadratic Programming (QP) is a type of mathematical optimization problem where the objective function is quadratic and the constraints are linear. The standard form of a QP problem is:

$$
\begin{aligned}
\min_x \quad & \frac{1}{2} x^T Q x + c^T x \\
\text{subject to} \quad & A x \leq b \\
\end{aligned}
$$

where:
- $ x \in \mathbb{R}^n $ is the vector of variables to be optimized,
- $ Q \in \mathbb{R}^{n \times n} $ is a symmetric positive definite matrix, ensuring the objective function is convex,
- $ c \in \mathbb{R}^n $ is the vector of linear coefficients,
- $ A \in \mathbb{R}^{m \times n} $ is the matrix representing the linear constraints,
- $ b \in \mathbb{R}^m $ is the vector representing the constraint bounds.

The objective function $ \frac{1}{2} x^T Q x + c^T x $ is quadratic because it includes a term involving the product of the variables (i.e., $ x^T Q x $). The constraints $ A x \leq b $ are linear inequalities.

### Active Set Method

The Active Set Method is an iterative algorithm used to solve QP problems, particularly those with linear constraints. The method involves iteratively adjusting a set of constraints that are "active" (binding) at the current solution estimate. The steps of the Active Set Method are as follows:

1. **Initialization**:
   - Start with an initial feasible solution $ x_0 $.
   - Identify the initial set of active constraints (constraints that are equalities at the current solution).

2. **Solving the Subproblem**:
   - Solve a QP subproblem with the current active constraints to find a search direction $ p $.

3. **Line Search**:
   - Determine the maximum step size $ \alpha $ along the search direction $ p $ that maintains feasibility with respect to all constraints.

4. **Updating**:
   - Update the solution: $ x_{k+1} = x_k + \alpha p $.
   - Update the set of active constraints.

5. **Termination**:
   - Check convergence criteria, such as the optimality conditions and feasibility. If the criteria are met, terminate the algorithm; otherwise, repeat the process.

The method relies on the fact that at the optimal solution, a subset of the constraints will be active. The algorithm iterates by exploring the feasible region defined by the constraints, updating the active set as necessary until convergence is achieved.

### Mathematical Details of the Active Set Method

The Active Set Method operates by solving a sequence of equality-constrained QP subproblems. At each iteration, the method focuses on a subset of constraints that are treated as equalities. The solution process can be described as follows:

1. **Subproblem Formulation**:
   - At iteration $ k $, let $ A_k $ represent the matrix of active constraints and $ b_k $ represent the corresponding bounds.
   - The subproblem is to minimize the quadratic objective subject to the equality constraints $ A_k x = b_k $.

2. **KKT Conditions**:
   - The Karush-Kuhn-Tucker (KKT) conditions are necessary for optimality in QP problems. For the active set $ A_k $, the KKT conditions are:

     $$
     \begin{aligned}
     & \nabla f(x_k) + A_k^T \lambda_k = 0 \\
     & A_k x_k = b_k
     \end{aligned}
     $$
     
     where $ \lambda_k $ are the Lagrange multipliers associated with the active constraints.

3. **Solving the KKT System**:
   - The KKT system can be written as:

     $$
     \begin{bmatrix}
     Q & A_k^T \\
     A_k & 0
     \end{bmatrix}
     \begin{bmatrix}
     p \\
     \lambda_k
     \end{bmatrix}
     =
     \begin{bmatrix}
     -\nabla f(x_k) \\
     0
     \end{bmatrix}
     $$

     Solving this system provides the search direction $ p $ and the Lagrange multipliers $ \lambda_k $.

4. **Step Size Determination**:
   - The step size $ \alpha $ is determined by maintaining feasibility with respect to all constraints:
     
     $$
     \alpha = \min \left(1, \min_{j \in \mathcal{I}} \left\{ \frac{b_j - A_j x_k}{A_j p} \mid A_j p > 0 \right\} \right)
     $$
     
     where $ \mathcal{I} $ is the set of inactive constraints.

5. **Updating**:
   - Update the solution: $ x_{k+1} = x_k + \alpha p $.
   - Update the active set by adding or removing constraints based on the Lagrange multipliers and step size calculations.

### Why We Didn't Implement the Active Set Method from Scratch

Implementing the Active Set Method from scratch is a complex and time-consuming task that requires handling various numerical challenges, such as ensuring numerical stability and efficiently solving linear systems. Instead, we utilized well-established libraries like SciPy and quadprog for the following reasons:

1. **Robustness**: These libraries are thoroughly tested and optimized for performance and accuracy. They handle edge cases and numerical issues that may arise during optimization.
2. **Efficiency**: Implementing a reliable QP solver from scratch requires significant effort in optimizing the code for performance. Libraries like SciPy and quadprog are optimized to deliver fast solutions.
3. **Focus on Problem Solving**: By leveraging these libraries, we can focus on the higher-level aspects of problem-solving, such as formulating the problem correctly and interpreting the results, rather than dealing with low-level implementation details.
4. **Reproducibility**: Using standard libraries ensures that the results are reproducible and comparable with other studies and applications that use the same libraries.

### Libraries Used and Their Methods

#### SciPy's SLSQP
The Sequential Least Squares Quadratic Programming (SLSQP) method, implemented in `scipy.optimize.minimize`, is a gradient-based optimization algorithm. It handles both equality and inequality constraints by transforming the constrained problem into a sequence of quadratic programming approximations. SLSQP is particularly useful for smooth optimization problems with a large number of constraints.

#### quadprog
The `quadprog` library provides an implementation of the active set method for solving QP problems. It iteratively adjusts the set of active constraints to find the optimal solution. This method is effective for problems with a moderate number of constraints and is known for its stability and efficiency.

### Implementation Details

The implementation follows these steps for each problem size:

1. **Problem Generation**: Generate a random matrix $ M $ and vector $ y $ such that $\|y\| \geq \|M\|_{2}$.
2. **Initial Guesses**: Define three different initial guesses for each problem size.
3. **Solvers Execution**: Solve the QP problems using both SLSQP and quadprog for each initial guess.
4. **Results Collection**: Collect the optimal solutions, objective values, execution times, and other relevant metrics.
5. **Summary**: Provide a summary of results for each run, including the number of iterations (for SLSQP), final iterate, stopping criteria (for SLSQP), and running time.

### Results and Summary

The results indicate that both SLSQP and quadprog effectively solve the QP problems for various sizes of $ M $ and $ y $. SLSQP provides additional insights, such as the number of iterations and stopping criteria, which are valuable for understanding the solver's behavior. Quadprog, while efficient, does not explicitly provide iteration counts.

Overall, the choice of solver may depend on specific problem requirements and the need for additional metrics. Both solvers demonstrate robustness and efficiency in handling the given QP problems.

### Batch of Five with Three Starting Points Each

In [ ]:
import numpy as np
from scipy.optimize import minimize
import quadprog
import pandas as pd
import time

# Function to generate random matrix M and vector y with given conditions
def generate_problem(m):
    M = np.random.randn(m, 2*m)
    # Compute the spectral norm of M
    spectral_norm = np.linalg.norm(M, 2)
    # Generate a random vector y with norm >= spectral norm of M
    y = np.random.randn(m)
    while np.linalg.norm(y) < spectral_norm:
        y = np.random.randn(m)
    return M, y, spectral_norm

# Function to solve with SLSQP
def solve_slsqp(M, y, x0):
    # Define the objective function
    def objective(x, M, y):
        return 0.5 * np.linalg.norm(M @ x - y)**2

    # Define the equality constraint (L1 norm constraint)
    def l1_constraint(x):
        return 1 - np.sum(np.abs(x))

    # Define the bounds for the variables
    bounds = [(-1, 1) for _ in range(len(x0))]

    # Set up the constraints
    constraints = {'type': 'ineq', 'fun': l1_constraint}

    # Solve the optimization problem using SLSQP
    start_time = time.time()
    result = minimize(objective, x0, args=(M, y), method='SLSQP', bounds=bounds, constraints=constraints)
    end_time = time.time()

    return result.x, result.fun, result.nit, end_time - start_time, result.message

# Function to solve with quadprog
def solve_quadprog(M, y, x0):
    def solve_qp(P, q, G, h, A=None, b=None):
        P = .5 * (P + P.T)  # make sure P is symmetric
        # Ensure P is positive definite by adding a small value to the diagonal
        P += np.eye(P.shape[0]) * 1e-8
        meq = 0 if A is None else A.shape[0]
        return quadprog.solve_qp(P, q, -G.T, -h, meq)[0]

    # Define the coefficients for the quadratic objective function
    P = M.T @ M
    q = -M.T @ y

    # Create the full P and q matrices including auxiliary variables z
    P_full = np.block([
        [P, np.zeros((2*m, 2*m))],
        [np.zeros((2*m, 2*m)), np.eye(2*m) * 1e-8]  # Add small values to ensure positive definiteness
    ])
    q_full = np.concatenate([q, np.zeros(2*m)])

    # Define the inequality constraints Gx <= h
    G = np.vstack([
        np.hstack([np.eye(2*m), -np.eye(2*m)]),    # x_i <= z_i
        np.hstack([-np.eye(2*m), -np.eye(2*m)]),   # -x_i <= z_i
        np.hstack([np.zeros((1, 2*m)), np.ones((1, 2*m))])  # sum(z) <= 1
    ]).astype(np.float64)
    h = np.concatenate([np.zeros(4*m), [1]])

    # Solve the quadratic programming problem
    start_time = time.time()
    x_opt = solve_qp(P_full, q_full, G, h)
    end_time = time.time()

    # Calculate the objective value
    objective_value = 0.5 * x_opt[:2*m].T @ (M.T @ M) @ x_opt[:2*m] + (-M.T @ y).T @ x_opt[:2*m]

    return x_opt[:2*m], objective_value, end_time - start_time

# Initialize lists to store results
results_slsqp = []
results_quadprog = []

# Solve for each case of m = 1, 2, 3, 4, 5 with three initial guesses each
for m in range(1, 6):
    M, y, spectral_norm = generate_problem(m)
    initial_guesses = [np.random.randn(2*m) for _ in range(3)]

    for x0 in initial_guesses:
        x_slsqp, obj_slsqp, nit_slsqp, time_slsqp, stop_criteria_slsqp = solve_slsqp(M, y, x0)
        x_quadprog, obj_quadprog, time_quadprog = solve_quadprog(M, y, x0)

        results_slsqp.append((m, x0, x_slsqp, obj_slsqp, nit_slsqp, time_slsqp, stop_criteria_slsqp, spectral_norm, np.linalg.norm(y)))
        results_quadprog.append((m, x0, x_quadprog, obj_quadprog, time_quadprog, spectral_norm, np.linalg.norm(y)))

# Create DataFrames for the results
df_slsqp = pd.DataFrame(results_slsqp, columns=['m', 'Initial Guess', 'Optimal Solution', 'Objective Value', 'Iterations', 'Execution Time', 'Stopping Criteria', 'Spectral Norm', 'Norm of y'])
df_quadprog = pd.DataFrame(results_quadprog, columns=['m', 'Initial Guess', 'Optimal Solution', 'Objective Value', 'Execution Time', 'Spectral Norm', 'Norm of y'])

# Display results
display("SLSQP Results:")
display(df_slsqp)
display("\nquadprog Results:")
display(df_quadprog)

# Provide summary for each run
for i in range(len(df_slsqp)):
    print(f"\nSummary for SLSQP run {i + 1}:")
    print(f"Number of iterations: {df_slsqp['Iterations'][i]}")
    print(f"Final iterate: {df_slsqp['Optimal Solution'][i]}")
    print(f"Stopping criteria: {df_slsqp['Stopping Criteria'][i]}")
    print(f"Running time: {df_slsqp['Execution Time'][i]:.6f} seconds")
    print(f"Spectral norm of M: {df_slsqp['Spectral Norm'][i]:.6f}")
    print(f"Norm of y: {df_slsqp['Norm of y'][i]:.6f}")

for i in range(len(df_quadprog)):
    print(f"\nSummary for quadprog run {i + 1}:")
    print(f"Final iterate: {df_quadprog['Optimal Solution'][i]}")
    print(f"Running time: {df_quadprog['Execution Time'][i]:.6f} seconds")
    print(f"Spectral norm of M: {df_quadprog['Spectral Norm'][i]:.6f}")
    print(f"Norm of y: {df_quadprog['Norm of y'][i]:.6f}")


'SLSQP Results:'

,m,Initial Guess,Optimal Solution,Objective Value,Iterations,Execution Time,Stopping Criteria,Spectral Norm,Norm of y
0,1,"[0.5152253047062878, -0.07799845745426785]","[0.0, 1.0]",0.004316,5,0.003675,Optimization terminated successfully,0.367180,0.460064
1,1,"[0.9956111213938071, -0.2794038062508357]","[-4.371708784147849e-13, 1.0]",0.004316,6,0.003647,Optimization terminated successfully,0.367180,0.460064
2,1,"[-0.7274184408642297, -1.0168477596302075]","[-1.734723475976807e-17, 1.0]",0.004316,5,0.002213,Optimization terminated successfully,0.367180,0.460064
3,2,"[0.10481434364036339, 0.11787520449584353, -0....","[-0.18122364513050054, -2.3554270598042562e-07...",0.703021,14,0.008331,Optimization terminated successfully,1.224610,2.011930
4,2,"[-1.2384353993190753, -1.351483910608706, -0.7...","[-0.18338061386030455, -2.474163490060573e-07,...",0.702971,21,0.013155,Optimization terminated successfully,1.224610,2.011930
5,2,"[-0.7676861063548793, -0.46590469577688376, 0....","[-0.1833078915280759, -9.520104153769347e-08, ...",0.702971,28,0.019567,Optimization terminated successfully,1.224610,2.011930
6,3,"[-1.5095884552564884, -2.9093883023430878, -0....","[1.1703482722011263e-06, 1.2551617174329662e-0...",1.830251,38,0.027736,Optimization terminated successfully,3.297551,3.333963
7,3,"[0.4383289056845291, -0.22211752425789097, 0.6...","[-2.6975424509380036e-07, 6.01679513433831e-08...",1.830207,94,0.065075,Optimization terminated successfully,3.297551,3.333963
8,3,"[2.068556978682236, -1.9735109813329161, -1.50...","[-2.142587490657154e-08, 1.2336475017803716e-0...",1.830206,79,0.044529,Optimization terminated successfully,3.297551,3.333963
9,4,"[-0.5672745530379648, 0.2618464238958286, 0.87...","[2.8042269582144186e-07, -0.2078369477674206, ...",7.671140,47,0.026701,Optimization terminated successfully,4.482608,4.560755


'\nquadprog Results:'

,m,Initial Guess,Optimal Solution,Objective Value,Execution Time,Spectral Norm,Norm of y
0,1,"[0.5152253047062878, -0.07799845745426785]","[4.6074255521944e-15, -1.0000000000000062]",0.236318,0.000197,0.367180,0.460064
1,1,"[0.9956111213938071, -0.2794038062508357]","[4.6074255521944e-15, -1.0000000000000062]",0.236318,0.000056,0.367180,0.460064
2,1,"[-0.7274184408642297, -1.0168477596302075]","[4.6074255521944e-15, -1.0000000000000062]",0.236318,0.000047,0.367180,0.460064
3,2,"[0.10481434364036339, 0.11787520449584353, -0....","[0.18290685408522966, 0.0, -1.7053025658242404...",2.097137,0.000062,1.224610,2.011930
4,2,"[-1.2384353993190753, -1.351483910608706, -0.7...","[0.18290685408522966, 0.0, -1.7053025658242404...",2.097137,0.000065,1.224610,2.011930
5,2,"[-0.7676861063548793, -0.46590469577688376, 0....","[0.18290685408522966, 0.0, -1.7053025658242404...",2.097137,0.000067,1.224610,2.011930
6,3,"[-1.5095884552564884, -2.9093883023430878, -0....","[-2.8033675305929997e-12, 3.1192569772487134e-...",6.313453,0.000091,3.297551,3.333963
7,3,"[0.4383289056845291, -0.22211752425789097, 0.6...","[-2.8033675305929997e-12, 3.1192569772487134e-...",6.313453,0.000062,3.297551,3.333963
8,3,"[2.068556978682236, -1.9735109813329161, -1.50...","[-2.8033675305929997e-12, 3.1192569772487134e-...",6.313453,0.000054,3.297551,3.333963
9,4,"[-0.5672745530379648, 0.2618464238958286, 0.87...","[6.850908729205685e-13, 0.20693838359039385, -...",3.459704,0.000061,4.482608,4.560755



Summary for SLSQP run 1:
Number of iterations: 5
Final iterate: [0. 1.]
Stopping criteria: Optimization terminated successfully
Running time: 0.003675 seconds
Spectral norm of M: 0.367180
Norm of y: 0.460064

Summary for SLSQP run 2:
Number of iterations: 6
Final iterate: [-4.37170878e-13  1.00000000e+00]
Stopping criteria: Optimization terminated successfully
Running time: 0.003647 seconds
Spectral norm of M: 0.367180
Norm of y: 0.460064

Summary for SLSQP run 3:
Number of iterations: 5
Final iterate: [-1.73472348e-17  1.00000000e+00]
Stopping criteria: Optimization terminated successfully
Running time: 0.002213 seconds
Spectral norm of M: 0.367180
Norm of y: 0.460064

Summary for SLSQP run 4:
Number of iterations: 14
Final iterate: [-1.81223645e-01 -2.35542706e-07 -1.15016228e-04 -8.18661574e-01]
Stopping criteria: Optimization terminated successfully
Running time: 0.008331 seconds
Spectral norm of M: 1.224610
Norm of y: 2.011930

Summary for SLSQP run 5:
Number of iterations: 21
Fi